Lab | Making predictions with logistic regression
In this lab, you will be using the Sakila database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented. We are asked to create a model to predict it. So we use the information we have from May 2005 to create the model.

* Instructions
* Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X).
* Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our TARGET (y) variable.
* Read the data into a Pandas dataframe. At this point you should have 1000 rows. Number of columns depends on the number of features you chose.
* Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.
* Create a logistic regression model to predict 'rented_in_may' from the cleaned data.
* Evaluate the results.

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error as MAS, mean_absolute_percentage_error as MAPE,  mean_squared_error as MSE, r2_score as R2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import getpass

In [45]:
mypass=getpass.getpass()
engine= 'mysql+pymysql://root:' + mypass + '@localhost/bank'

I will create a query to establish a scoring system for movies based on whether they have prolific actors.

In [46]:
film_score=pd.read_sql('''select sum(a.score) as film_score, f.film_id from 
                        (select count(*) as score, a.actor_id from sakila.actor a 
                        join sakila.film_actor fm on a.actor_id=fm.actor_id 
                        group by a.actor_id) a 
                        join sakila.film_actor fa on a.actor_id = fa.actor_id
                        right join sakila.film f on fa.film_id=f.film_id
                        group by f.film_id''', engine)
film_score.head()

,film_score,film_id
0,278.0,1
1,109.0,2
2,136.0,3
3,141.0,4
4,141.0,5


In [47]:
sakila=pd.read_sql('''select f.title, f.film_id, c.name as category, f.rental_rate, f.length, f.rating, f.special_features from sakila.film f 
                    join sakila.film_category fc on f.film_id=fc.film_id 
                    join sakila.category c on fc.category_id=c.category_id 
                    join sakila.language l on f.language_id=l.language_id''', engine)
sakila.sort_values('film_id').head()

,title,film_id,category,rental_rate,length,rating,special_features
305,ACADEMY DINOSAUR,1,Documentary,0.99,86,PG,"Deleted Scenes,Behind the Scenes"
638,ACE GOLDFINGER,2,Horror,4.99,48,G,"Trailers,Deleted Scenes"
306,ADAPTATION HOLES,3,Documentary,2.99,50,NC-17,"Trailers,Deleted Scenes"
639,AFFAIR PREJUDICE,4,Horror,2.99,117,G,"Commentaries,Behind the Scenes"
435,AFRICAN EGG,5,Family,2.99,130,G,Deleted Scenes


In [48]:
rented_in_may=pd.read_sql('''select f.film_id,
                            max(month(rental_date)=5 and year(rental_date)=2005) as rented_in_may from sakila.rental r 
                            join sakila.inventory i on r.inventory_id=i.inventory_id 
                            right join sakila.film f on f.film_id=i.film_id
                            group by f.film_id''', engine)
rented_in_may.head()

,film_id,rented_in_may
0,1,1.0
1,2,0.0
2,3,1.0
3,4,1.0
4,5,1.0


In [49]:
data=pd.concat([i.sort_values('film_id').reset_index(drop=True).drop('film_id',axis=1) for i in [rented_in_may,film_score,sakila]],axis=1).fillna(0)
data.index=sakila['film_id'].sort_values()
data.head()

,rented_in_may,film_score,title,category,rental_rate,length,rating,special_features
film_id,,,,,,,,
1,1.0,278.0,ACADEMY DINOSAUR,Documentary,0.99,86,PG,"Deleted Scenes,Behind the Scenes"
2,0.0,109.0,ACE GOLDFINGER,Horror,4.99,48,G,"Trailers,Deleted Scenes"
3,1.0,136.0,ADAPTATION HOLES,Documentary,2.99,50,NC-17,"Trailers,Deleted Scenes"
4,1.0,141.0,AFFAIR PREJUDICE,Horror,2.99,117,G,"Commentaries,Behind the Scenes"
5,1.0,141.0,AFRICAN EGG,Family,2.99,130,G,Deleted Scenes


I will create a column to determine if the film has special features

In [50]:
for i in ['Trailers','Commentaries','Deleted Scenes','Behind the Scenes']:
    data[i]= [int(i in data.special_features[c]) for c in data.index]

data.head()



,rented_in_may,film_score,title,category,rental_rate,length,rating,special_features,Trailers,Commentaries,Deleted Scenes,Behind the Scenes
film_id,,,,,,,,,,,,
1,1.0,278.0,ACADEMY DINOSAUR,Documentary,0.99,86,PG,"Deleted Scenes,Behind the Scenes",0,0,1,1
2,0.0,109.0,ACE GOLDFINGER,Horror,4.99,48,G,"Trailers,Deleted Scenes",1,0,1,0
3,1.0,136.0,ADAPTATION HOLES,Documentary,2.99,50,NC-17,"Trailers,Deleted Scenes",1,0,1,0
4,1.0,141.0,AFFAIR PREJUDICE,Horror,2.99,117,G,"Commentaries,Behind the Scenes",0,1,0,1
5,1.0,141.0,AFRICAN EGG,Family,2.99,130,G,Deleted Scenes,0,0,1,0


In [51]:
X= data.drop(['rented_in_may','special_features','title'], axis=1)
y=data['rented_in_may']

In [69]:
numerical, categorical= X._get_numeric_data(), X.select_dtypes('object')

(1000, 2)

In [53]:
onehot=OneHotEncoder(drop='first').fit(categorical)
encoded=onehot.transform(categorical)
encoded=pd.DataFrame(encoded.toarray(),columns=onehot.get_feature_names_out(),index=data.index)
encoded

,category_Animation,category_Children,category_Classics,category_Comedy,category_Documentary,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R
film_id,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [54]:
minmax=MinMaxScaler().fit(numerical)
scaled=minmax.transform(numerical)
scaled=pd.DataFrame(scaled,columns=numerical.columns,index=data.index)
scaled

,film_score,rental_rate,length,Trailers,Commentaries,Deleted Scenes,Behind the Scenes
film_id,,,,,,,
1,0.631818,0.0,0.287770,0.0,0.0,1.0,1.0
2,0.247727,1.0,0.014388,1.0,0.0,1.0,0.0
3,0.309091,0.5,0.028777,1.0,0.0,1.0,0.0
4,0.320455,0.5,0.510791,0.0,1.0,0.0,1.0
5,0.320455,0.5,0.604317,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
996,0.297727,0.0,0.985612,1.0,0.0,0.0,1.0
997,0.384091,0.0,0.956835,1.0,0.0,0.0,1.0
998,0.390909,0.0,0.424460,0.0,0.0,1.0,0.0


In [70]:
finaldf=pd.concat([scaled,encoded],axis=1)
finaldf.shape

(1000, 26)

In [71]:
X_train,X_test,y_train,y_test=train_test_split(finaldf,y)

In [72]:

LR= LogisticRegression(solver='lbfgs').fit(X_train,y_train)
predictions=LR.predict(X_test)



In [81]:
for i in ['MAPE','MAS','MSE','R2']:
    if i =='MSE':
        print('R'+i,'is equal',round(np.sqrt(eval(i + '(y_test,predictions)')),2))
    print(i,'is equal to',round(eval(i + '(y_test,predictions)'),2))

MAPE is equal to 1170935903116329.0
MAS is equal to 0.28
RMSE is equal 0.53
MSE is equal to 0.28
R2 is equal to -0.46
